In [ ]:
import sys
import os
import json
import logging  # Import logging module
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout,
    QTableWidget, QTableWidgetItem, QPushButton, QSplitter, QListWidget, QLabel,
    QMessageBox, QStatusBar, QDialog, QDialogButtonBox, QProgressBar
)
from PyQt5.QtCore import Qt, QUrl, QTimer, pyqtSignal
from PyQt5.QtWebEngineWidgets import QWebEngineView, QWebEnginePage
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class CustomWebEnginePage(QWebEnginePage):
    def javaScriptConsoleMessage(self, level, message, lineNumber, sourceId):
        print(f"JavaScript console message: {level=}, {message=}, {lineNumber=}, {sourceId=}")
        logging.debug(f"JavaScript console message: level={level}, message={message}, lineNumber={lineNumber}, sourceId={sourceId}") # Log JS console messages

class MainWindow(QMainWindow):
    auto_collection_finished_signal = pyqtSignal() # Signal for auto-collection finish

    def __init__(self):
        super().__init__()
        self.login_data = self.load_login_data()
        self.site_url_patterns = self.load_site_url_patterns() # Load site URL patterns
        self.web_view = QWebEngineView() # Web view를 __init__에서 초기화
        self.web_view.setPage(CustomWebEnginePage(self.web_view))
        self.web_view.page().profile().setHttpUserAgent("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36") # Chrome User-Agent 설정
        self.category_url_list_widget = QListWidget() # Category URLs 표시 위젯 # Initialize here
        self.product_url_list_widget = QListWidget() # Product URLs 표시 위젯 # Initialize here
        self.table_widget = QTableWidget() # Initialize table_widget here, before initUI uses it
        self.initUI()
        self.login_attempt_count = 0
        self.is_login_process = False
        self.script_folder = "site_scripts"
        self.is_collecting = False
        self.collected_urls = set()
        self.current_site_config = None # 현재 사이트 설정을 저장할 변수
        self.auto_collection_finished_signal.connect(self.on_auto_collection_finished) # Connect signal


    def initUI(self):
        self.setWindowTitle("Sample Crawler UI")
        self.setGeometry(100, 100, 1200, 800)

        self.status_bar = QStatusBar()
        self.setStatusBar(self.status_bar)
        self.status_bar.showMessage("Ready")

        self.progress_bar = QProgressBar() # Progress bar 추가
        self.progress_bar.setMaximum(0) # Indeterminate progress bar
        self.progress_bar.setVisible(False) # Initially hidden
        self.status_bar.addPermanentWidget(self.progress_bar) # Add to status bar

        main_widget = QWidget()
        main_layout = QVBoxLayout(main_widget)

        top_layout = QHBoxLayout()

        load_sites_button = QPushButton("Load Sites")
        load_sites_button.clicked.connect(self.load_sites)

        open_site_button = QPushButton("Open Selected Site")
        open_site_button.clicked.connect(self.open_selected_site)

        login_button = QPushButton("Login")
        login_button.clicked.connect(self.login_to_site)

        current_category_button = QPushButton("Add Current Category URL") # 버튼 텍스트 변경
        current_category_button.clicked.connect(self.show_current_category)

        stop_collection_button = QPushButton("Stop Collection")
        stop_collection_button.clicked.connect(self.stop_collection)

        reset_collection_button = QPushButton("Reset Collected Items") # 버튼 텍스트 변경
        reset_collection_button.clicked.connect(self.reset_collection_items)

        auto_collection_button = QPushButton("Start Auto Collection") # 버튼 텍스트 변경
        auto_collection_button.clicked.connect(self.start_auto_collection)


        # 버튼들을 리스트로 관리
        buttons = [
            load_sites_button, open_site_button, login_button,
            current_category_button, stop_collection_button, reset_collection_button,
            auto_collection_button
        ]

        # 버튼들의 최대 가로 길이를 계산
        max_button_width = 0
        for button in buttons:
            max_button_width = max(max_button_width, button.sizeHint().width())

        for button in buttons:
            top_layout.addWidget(button)

        main_layout.addLayout(top_layout)

        middle_layout = QHBoxLayout() # 중단 레이아웃 (수평)
        middle_layout.setSpacing(5) # 위젯 사이 공백 최소화 <- 변경 부분

        self.site_list_widget = QListWidget()
        self.site_list_widget.setMaximumWidth(600) # 사이트 리스트 최대 가로 길이 설정 (버튼 가로 길이의 2배)
        self.site_list_widget.setMaximumHeight(400) # 로드사이트 위젯 세로 사이즈 400으로 설정
        middle_layout.addWidget(self.site_list_widget) # 사이트 목록 위젯을 중단 레이아웃에 추가

        url_list_container = self.create_url_list_widget_container() # URL 리스트 컨테이너 생성
        middle_layout.addWidget(url_list_container) # URL 리스트 컨테이너를 중단 레이아웃에 추가

        main_layout.addLayout(middle_layout) # 중단 레이아웃을 메인 레이아웃에 추가


        h_splitter = QSplitter(Qt.Horizontal) # 수평 스플리터
        h_splitter.addWidget(self.web_view) # 웹 뷰 추가
        h_splitter.addWidget(self.table_widget) # 테이블 위젯 추가


        self.web_view.loadStarted.connect(self.on_load_started) # loadStarted 시그널 연결
        self.web_view.loadProgress.connect(self.on_load_progress) # loadProgress 시그널 연결
        self.web_view.loadFinished.connect(self.on_load_finished) # loadFinished 시그널 연결

        h_splitter.setSizes([int(self.width() * 0.7), int(self.width() * 0.3)]) # 웹뷰, 테이블 영역 비율 조정

        main_layout.addWidget(h_splitter) # 메인 레이아웃에 수평 스플리터 추가
        self.setCentralWidget(main_widget)

        self._setup_table_widget() # 테이블 위젯 내용 설정 분리


    def create_url_list_widget_container(self):
        container_widget = QWidget()
        container_layout = QHBoxLayout(container_widget)
        container_layout.setSpacing(5) # URL 리스트 컨테이너 위젯 사이 공백 최소화 <- 변경 부분

        # category_label = QLabel("Category URLs") # 라벨 제거
        # product_label = QLabel("Product URLs")   # 라벨 제거

        # container_layout.addWidget(category_label) # 라벨 제거
        container_layout.addWidget(self.category_url_list_widget) # Category URL 리스트 추가
        # container_layout.addWidget(product_label)  # 라벨 제거
        container_layout.addWidget(self.product_url_list_widget)  # Product URL 리스트 추가
        container_widget.setMaximumHeight(400)  # 최대 세로 길이 400으로 설정
        container_widget.setMaximumWidth(600)  # URL 리스트 컨테이너 최대 가로 길이 설정 (두 위젯을 나란히 놓기 위해 늘림)
        return container_widget

    def _setup_table_widget(self): # 테이블 위젯 내용 설정 메소드
        self.table_widget.setRowCount(10)

        header_labels = ["원상품코드", "상품코드", "원상품명", "변경상품명", "스마트스토어전용상품명", "쿠팡전용상품명",
                         "상품명길이", "판매단위", "마이링크", "주키워드", "부키워드", "세키워드", "네이버쇼핑사전등록태그",
                         "스마트스토어 태그", "실제등록상품명", "등록불가단어", "원산지", "제조사", "수입사", "브랜드",
                         "모델명", "과세여부", "나이제한", "제조일", "유효기간", "홍보문구", "공급원가", "가격기준",
                         "가격요율", "판매금액", "우선판매금액", "옵션입력", "옵션명", "옵션정보", "옵션개수", "옵션분할명",
                         "재고수량", "원대표이미지", "원추가이미지1", "원추가이미지2", "원추가이미지3", "원추가이미지4",
                         "원추가이미지5", "대표이미지", "추가이미지1", "추가이미지2", "추가이미지3", "추가이미지4",
                         "추가이미지5", "원상세설명", "상세설명", "상세설명특이사항상단", "상세설명특이사항하단", "최종상세설명",
                         "옥션", "지마켓", "11번가", "11번가상품속성명", "11번가상품세부속성명", "11번가추천옵션명", "인터파크",
                         "스마트스토어", "쿠팡", "쿠팡추천옵션명", "쿠팡검색옵션명", "옥션CAT", "지마켓CAT", "11번가CAT",
                         "인터파크CAT", "스마트스토어CAT", "쿠팡CAT", "ESM옥션카테고리", "ESM지마켓카테고리", "ESM옥션추천옵션명",
                         "ESM지마켓추천옵션명", "티몬", "위메프", "롯데ON", "롯데ON구매옵션", "톡스토어", "카페24", "멸치쇼핑",
                         "72TIME", "에이블리", "특이사항", "인증대상여부", "인증품목", "인증번호", "인증기관", "인증상호",
                         "인증일자", "인증제조국", "인증제조사", "인증모델명", "인증상태", "최저가준수", "소비자가", "상품상태",
                         "단위", "규격", "ISBN", "최소구매수량", "이셀러스요약정보상품군코드", "스마트스토어상품속성", "쿠팡검색필터",
                         "쿠팡전용필수옵션", "공급사카테고리", "수집1", "수집2", "수집3", "수집4", "수집5", "수집6", "수집7",
                         "수집8", "수집9", "수집10", "수집배송비", "편집1", "편집2", "편집3", "편집4", "편집5", "편집6",
                         "편집7", "편집8", "편집9", "편집10", "단독옵션일괄변경여부", "비고", "경로코드", "원본경로"]
        self.table_widget.setColumnCount(len(header_labels))
        self.table_widget.setHorizontalHeaderLabels(header_labels)

        for row in range(10):
            self.table_widget.setItem(row, 0, QTableWidgetItem(f"상품명 {row+1}"))
            self.table_widget.setItem(row, 1, QTableWidgetItem(str((row+1)*1000)))
            self.table_widget.setItem(row, 2, QTableWidgetItem(str(row+1)))
            self.table_widget.setItem(row, 3, QTableWidgetItem("기타 정보"))


    def load_login_data(self):
        try:
            df = pd.read_excel("login_data.xlsx")
            login_dict = {}
            for index, row in df.iterrows():
                site_url = row['사이트목록']
                login_dict[site_url] = {
                    '아이디': row['아이디'],
                    '비밀번호': row['비밀번호'],
                    '로그인페이지': row['로그인페이지'] if pd.notnull(row['로그인페이지']) else None,
                    '로그인후페이지': row['로그인후페이지'] if pd.notnull(row['로그인후페이지']) else None,
                    'username_selector': row['username_selector'] if pd.notnull(row['username_selector']) else None,
                    'password_selector': row['password_selector'] if pd.notnull(row['password_selector']) else None,
                    'login_button_selector': row['login_button_selector'] if pd.notnull(row['login_button_selector']) else None,
                    'login_script_file': row['login_script_file'] if pd.notnull(row['login_script_file']) else None,
                }
            return login_dict
        except FileNotFoundError:
            error_msg = "login_data.xlsx 파일을 찾을 수 없습니다."
            QMessageBox.critical(self, "Error", error_msg)
            logging.error(error_msg) # Log error
            return {}
        except Exception as e:
            error_msg = f"login_data.xlsx 파일을 읽는 중 오류 발생: {e}"
            QMessageBox.critical(self, "Error", error_msg)
            logging.exception(error_msg) # Log exception with traceback
            return {}

    def load_site_url_patterns(self):
        try:
            with open("site_url_patterns.json", "r", encoding="utf-8") as f:
                return json.load(f)
        except FileNotFoundError:
            error_msg = "site_url_patterns.json 파일을 찾을 수 없습니다."
            QMessageBox.critical(self, "Error", error_msg)
            logging.error(error_msg) # Log error
            return {}
        except json.JSONDecodeError:
            error_msg = "site_url_patterns.json 파일 형식이 잘못되었습니다."
            QMessageBox.critical(self, "Error", error_msg)
            logging.error(error_msg) # Log error
            return {}
        except Exception as e:
            error_msg = f"site_url_patterns.json 파일을 로드 중 오류 발생: {e}"
            QMessageBox.critical(self, "Error", error_msg)
            logging.exception(error_msg) # Log exception with traceback
            return {}

    def load_sites(self):
        self.site_list_widget.clear()
        try:
            with open("site_list.txt", "r", encoding="utf-8") as f:
                for line in f:
                    site = line.strip()
                    if site:
                        self.site_list_widget.addItem(site)
            if not self.site_list_widget.count():
                QMessageBox.information(self, "Info", "사이트 목록 파일에 사이트 정보가 없습니다.")
        except FileNotFoundError:
            error_msg = "site_list.txt 파일을 찾을 수 없습니다."
            QMessageBox.critical(self, "Error", error_msg)
            logging.error(error_msg) # Log error
        except Exception as e:
            error_msg = f"사이트 목록 파일을 읽는 중 오류 발생: {e}"
            QMessageBox.critical(self, "Error", error_msg)
            logging.exception(error_msg) # Log exception with traceback

    def open_selected_site(self):
        current_item = self.site_list_widget.currentItem()
        if current_item:
            url_text = current_item.text().strip()
            if not url_text.startswith(("http://", "https://")):
                url_text = "https://" + url_text
            url = QUrl(url_text)
            if url.isValid():
                logging.info(f"Loading site: {url_text}")
                self.web_view.load(url)
            else:
                QMessageBox.warning(self, "Warning", f"유효하지 않은 URL입니다: {url_text}")
                logging.warning(f"Invalid URL: {url_text}")
        else:
            QMessageBox.information(self, "Info", "사이트를 선택해주세요.")

    def login_to_site(self):
        current_item = self.site_list_widget.currentItem()
        if not current_item:
            QMessageBox.information(self, "Info", "로그인할 사이트를 먼저 선택해주세요.")
            return

        site_url_base = current_item.text().strip()
        if not site_url_base.startswith(("http://", "https://")):
            site_url_base = "https://" + site_url_base

        login_info = self.login_data.get(site_url_base)
        if not login_info:
            QMessageBox.warning(self, "Warning", f"{site_url_base}에 대한 로그인 정보가 login_data.xlsx에 없습니다.")
            logging.warning(f"No login info found for {site_url_base} in login_data.xlsx")
            return

        login_page_url = login_info.get('로그인페이지')
        if not login_page_url:
            QMessageBox.warning(self, "Warning", f"{site_url_base}의 로그인 페이지 URL이 login_data.xlsx에 설정되지 않았습니다.")
            logging.warning(f"Login page URL not set for {site_url_base} in login_data.xlsx")
            return

        self.web_view.load(QUrl(login_page_url))
        self.status_bar.showMessage(f"로그인 페이지 {login_page_url} 로딩 중... (Attempt {self.login_attempt_count + 1})")
        logging.info(f"Loading login page {login_page_url} for {site_url_base}, attempt {self.login_attempt_count + 1}")
        self.login_attempt_count += 1
        self.is_login_process = True

        self.current_login_info = login_info
        self.web_view.login_site_url_base = site_url_base

    def load_login_script(self, site_url_base):
        script_filename = site_url_base.replace("https://", "").replace("http://", "").replace(".", "-") + ".js"
        script_path = os.path.join(self.script_folder, script_filename)

        try:
            with open(script_path, "r", encoding="utf-8") as f:
                script_content = f.read()
                logging.info(f"Loaded login script from {script_path}")
                return script_content
        except FileNotFoundError:
            logging.warning(f"Login script file not found: {script_path}")
            return None
        except Exception as e:
            logging.error(f"Error reading login script file {script_path}: {e}")
            return None


    def on_load_started(self):
        self.status_bar.showMessage("Loading...")
        self.setWindowTitle(f"Loading... - Sample Crawler UI")
        self.progress_bar.setVisible(True) # Show progress bar

    def on_load_progress(self, progress):
        self.status_bar.showMessage(f"Loading... {progress}% ")

    def on_load_finished(self, ok):
        self.progress_bar.setVisible(False) # Hide progress bar
        if ok:
            url = self.web_view.url().toString()
            self.setWindowTitle(f"{self.web_view.title()} - Sample Crawler UI")
            logging.info(f"Page loaded successfully: {url}")

            if url.startswith("https://www.dzb.co.kr/goods/populate.php"): # populate.php 페이지 감지
                logging.warning("Detected and stopped loading of populate.php page.")
                self.web_view.stop() # 페이지 로드 강제 중단
                return # 더 이상 로그인 시도 등 후속 처리 진행 X, 함수 종료


            if self.is_login_process:
                login_info = self.current_login_info
                site_url_base = self.web_view.login_site_url_base

                logging.info(f"Attempting login for {site_url_base}, attempt {self.login_attempt_count}")

                if login_info:
                    username = login_info['아이디']
                    password = login_info['비밀번호']
                    username_selector = login_info.get('username_selector')
                    password_selector = login_info.get('password_selector')
                    login_button_selector = login_info.get('login_button_selector')
                    login_script_file = login_info.get('login_script_file')

                    if login_script_file:
                        script_path = os.path.join(self.script_folder, login_script_file)
                        try:
                            with open(script_path, "r", encoding="utf-8") as f:
                                script_content = f.read()
                                script = script_content.format(
                                    username=username,
                                    password=password,
                                    login_attempt_count=self.login_attempt_count,
                                    site_url_base=site_url_base,
                                    username_selector=username_selector,
                                    password_selector=password_selector,
                                    login_button_selector=login_button_selector,
                                )
                                QTimer.singleShot(1000, lambda: self.web_view.page().runJavaScript(script))
                                self.status_bar.showMessage(f"{site_url_base} 로그인 시도 중... (Attempt {self.login_attempt_count}) - Script File")
                                logging.info(f"Executing login script from file for {site_url_base}, attempt {self.login_attempt_count}, script file: {login_script_file}")

                        except FileNotFoundError:
                            error_msg = f"{site_url_base}에 대한 로그인 스크립트 파일을 찾을 수 없습니다: {script_path}"
                            QMessageBox.warning(self, "Warning", error_msg)
                            self.status_bar.showMessage(f"{site_url_base} 로그인 스크립트 파일 없음 (Attempt {self.login_attempt_count}): {script_path}")
                            logging.warning(error_msg)

                    else:
                        script = f"""
                            (function() {{
                                let usernameField = document.querySelector('{username_selector}');
                                let passwordField = document.querySelector('{password_selector}');
                                let loginButton = document.querySelector('{login_button_selector}');

                                console.log('Login attempt: {{login_attempt_count}}');
                                console.log('Username field selector: {username_selector}, Password field selector: {password_selector}, Login button selector: {login_button_selector}');

                                if (usernameField) {{
                                    console.log('Username field found:', usernameField);
                                    usernameField.value = '{{username}}';
                                    console.log('Username field value set.');
                                }} else {{
                                    console.error('Username field not found for {{site_url_base}}');
                                }}

                                if (passwordField) {{
                                    console.log('Password field found:', passwordField);
                                    passwordField.value = '{{password}}';
                                    console.log('Password field value set.');
                                }} else {{
                                    console.error('Password field not found for {{site_url_base}}');
                                }}

                                if (loginButton) {{
                                    let loginFunc = loginButton.onclick;
                                    if (loginFunc) {{
                                        console.log('Login button onclick function found:', loginFunc);
                                        loginFunc.call(loginButton);
                                        console.log('Login button onclick function executed.');
                                    }} else {{
                                        console.warn('Login button onclick function NOT found. Trying button click fallback.');
                                        loginButton.click();
                                        console.log('Login button clicked (fallback).');
                                    }}

                                }} else {{
                                    console.warn('Login button not found for {{site_url_base}}. Form submission might be required.');
                                }}
                            }})();
                        """
                        script = script.format(
                            username=username,
                            password=password,
                            login_attempt_count=self.login_attempt_count,
                            site_url_base=site_url_base,
                            username_selector=username_selector if username_selector else '#member_id',
                            password_selector=password_selector if password_selector else '#member_passwd',
                            login_button_selector=login_button_selector if login_button_selector else 'a.-btn.-block.-xl.-black',
                        )
                        QTimer.singleShot(1000, lambda: self.web_view.page().runJavaScript(script))
                        self.status_bar.showMessage(f"{site_url_base} 로그인 시도 중... (Attempt {self.login_attempt_count}) - Default Script")
                        logging.info(f"Executing default login script for {site_url_base}, attempt {self.login_attempt_count}")

                    self.is_login_process = False
                    del self.current_login_info
                    del self.web_view.login_site_url_base
                else:
                    self.status_bar.showMessage("Load finished")
        else:
            self.status_bar.showMessage("Load failed")
            self.setWindowTitle(f"Load Failed - Sample Crawler UI")
            logging.error(f"Page load failed: {self.web_view.url().toString()}")

    def show_current_category(self):
        current_url = self.web_view.url().toString()
        if current_url:
            self.category_url_list_widget.addItem(current_url) # url_dialog_1 대신 category_url_list_widget 에 추가
            self.status_bar.showMessage(f"Current URL '{current_url}' added to Category URLs list.")
            logging.info(f"Current URL added to Category URLs list: {current_url}")
        else:
            self.status_bar.showMessage("Could not get current URL.")
            logging.warning("Could not get current URL when 'Add Current Category URL' button clicked.")

    def stop_collection(self):
        if self.is_collecting:
            self.is_collecting = False
            self.status_bar.showMessage("Collection stopped. Will stop after current page.")
            logging.info("Collection stopped by user.")
        else:
            self.status_bar.showMessage("Not collecting.")
            logging.info("Stop collection button clicked, but not collecting.")

    def reset_collection_items(self):
        self.category_url_list_widget.clear() # url_dialog_1 대신 category_url_list_widget clear
        self.product_url_list_widget.clear() # url_dialog_2 대신 product_url_list_widget clear
        self.collected_urls = set()
        self.status_bar.showMessage("Collected items reset.")
        logging.info("Collected items reset by user.")

    def start_auto_collection(self):
        current_item = self.site_list_widget.currentItem()
        if not current_item:
            QMessageBox.information(self, "Info", "Please select a site first.")
            return

        site_url_full = current_item.text().strip()
        site_domain = QUrl(site_url_full).host()

        self.current_site_config = self.site_url_patterns.get(site_domain) # Get site config
        if not self.current_site_config:
            QMessageBox.warning(self, "Warning", f"No URL patterns found for {site_domain}.")
            logging.warning(f"No URL patterns found for {site_domain} in site_url_patterns.json")
            return

        start_category_code = self.current_site_config.get('start_category_code', "106") # 설정 파일에서 시작 카테고리 코드 가져오기, 없으면 "106" 기본값 사용
        start_page = self.current_site_config.get('start_page', 1) # 설정 파일에서 시작 페이지 가져오기, 없으면 1 기본값 사용
        category_url = self.current_site_config['category_list_url'].format(category_code=start_category_code, page_number=start_page) # 시작 페이지 적용

        self.collected_urls = set()
        self.category_url_list_widget.clear() # url_dialog_1 대신 category_url_list_widget clear
        self.product_url_list_widget.clear() # url_dialog_2 대신 product_url_list_widget clear
        self.is_collecting = True
        self.status_bar.showMessage("Auto collection started...")
        logging.info(f"Auto collection started for {site_domain}, starting URL: {category_url}")
        self.collect_product_urls_from_page(category_url, self.current_site_config) # 사이트 설정 전달

    def collect_product_urls_from_page(self, category_url, site_config):
        if not self.is_collecting:
            self.status_bar.showMessage("Collection stopped.")
            logging.info("Auto collection stopped.")
            self.auto_collection_finished_signal.emit() # Emit finished signal
            return

        self.status_bar.showMessage(f"Analyzing page: {category_url}")
        logging.info(f"Analyzing page for product URLs: {category_url}")

        product_link_selectors = site_config.get('product_link_selectors', [
            'ul.prdList > li a[href]',
            '.prd-item a[href]',
            'li.item a[href]',
            'div.list a[href]',
            'a.product-link[href]'
        ])
        next_page_selectors = site_config.get('next_page_selectors', [
            'a.next',
            '.next a',
            'a.page-next',
            '.pagination > li:last-child > a'
        ])

        js_code = f"""
        (function() {{
            let productUrls = [];
            let productElements = [];
            let selectors = {json.dumps(product_link_selectors)};

            for (const selector of selectors) {{
                const elements = document.querySelectorAll(selector);
                if (elements.length > 0) {{
                    productElements = elements;
                    console.log('Product link selector "' + selector + '" found elements.'); // 수정: selector 변수 사용
                    break; // Stop after finding elements with the first selector
                }} else {{
                    console.log('Product link selector "' + selector + '" found no elements.'); // 수정: selector 변수 사용
                }}
            }}


            if (!productElements.length) {{
                console.log('No product links found with any selector. Check selectors in site_url_patterns.json.');
                return {{ productUrls: [], nextPageUrl: null }};
            }}

            productElements.forEach(element => {{
                let url = element.href;
                if (url && !url.startsWith('javascript:')) {{
                    productUrls.push(url);
                }}
            }});
            console.log('Found product URLs:', productUrls.length);

            let nextPageUrl = null; // *** IMPORTANT CHANGE: DO NOT EXTRACT nextPageUrl from website ***
            let nextPageElement = null; // No longer need to find nextPageElement

            console.log('Next page URL: (Manual Construction)', nextPageUrl); // Indicate manual construction in log
            return {{ productUrls: productUrls, nextPageUrl: nextPageUrl }};
        }})();
        """

        def handle_results(result):
            if not self.is_collecting:
                self.status_bar.showMessage("Collection stopped.")
                logging.info("Auto collection stopped during page analysis.")
                self.auto_collection_finished_signal.emit() # Emit finished signal
                return

            if result:
                product_urls_page = result['productUrls']
                next_page_url = result['nextPageUrl'] # This will always be None now

                new_urls_count = 0
                for url in product_urls_page:
                    if url not in self.collected_urls:
                        self.collected_urls.add(url)
                        self.category_url_list_widget.addItem(url) # url_dialog_1 대신 category_url_list_widget 에 추가
                        new_urls_count += 1

                self.status_bar.showMessage(f"Collected {new_urls_count} new product URLs from page. Total: {len(self.collected_urls)} (Check Category URLs and Product URLs lists)") # 메시지 변경
                logging.info(f"Collected {new_urls_count} new product URLs from page. Total: {len(self.collected_urls)}")

                if  self.is_collecting: # next_page_url condition removed - always try to go to next page if collecting
                    if next_page_url: # This condition will always be false now, but keep it for potential future use
                        if not next_page_url.startsWith(("http://", "https://")):
                            base_url = QUrl(category_url)
                            next_page_url = base_url.resolved(QUrl(next_page_url)).toString() # 수정: next_page_url 로 resolve
                        logging.info(f"Next page URL found: {next_page_url}") # This log will not be reached in current setup

                    current_page_number = int(QUrl(category_url).query().split(site_config.get('page_number_param', "page") + "=")[1].split("&")[0]) if site_config.get('page_number_param', "page") + "=" in QUrl(category_url).query() else site_config.get('start_page', 1) # 페이지 번호 파라미터 이름 설정 파일에서 가져오기
                    next_page_number = current_page_number + site_config.get('page_increment', 1) # 페이지 증가 값 설정 파일에서 가져오기

                    next_page_url_formatted = site_config['page_url_format'].format(
                        category_code=site_config.get('start_category_code', "106"), # 카테고리 코드 설정 파일에서 가져오기
                        page_number=next_page_number
                    )

                    QTimer.singleShot(2000, lambda: self.web_view.load(QUrl(next_page_url_formatted))) # 딜레이 2초로 증가
                else:
                    self.is_collecting = False
                    self.status_bar.showMessage("Auto collection finished. (Check Category URLs and Product URLs lists)") # 메시지 변경
                    logging.info("Auto collection finished.")
                    self.auto_collection_finished_signal.emit() # Emit finished signal
            else:
                self.is_collecting = False
                self.status_bar.showMessage("Failed to collect product URLs.")
                logging.error("Failed to collect product URLs from page.")
                self.auto_collection_finished_signal.emit() # Emit finished signal


    def on_auto_collection_finished(self):
        self.progress_bar.setMaximum(100) # Set to determinate to hide indeterminate animation
        self.progress_bar.setValue(100) # Fill progress bar
        QTimer.singleShot(1000, lambda: self.progress_bar.setVisible(False)) # Hide after 1 sec


def main():
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    if not os.path.exists("site_scripts"):
        os.makedirs("site_scripts")

    dzb_script_path = os.path.join("site_scripts", "www-dzb-co-kr.js")
    if not os.path.exists(dzb_script_path):
        with open(dzb_script_path, "w", encoding="utf-8") as f:
            f.write("""
(function() {{
    let usernameField = document.querySelector('#loginId');
    let passwordField = document.querySelector('#loginPwd');
    let loginButton = document.querySelector('#formLogin > div.login > button');

    console.log('Login attempt: {login_attempt_count}');
    console.log('Username field selector: #loginId, Password field selector: #loginPwd, Login button selector: #formLogin > div.login > button');

    if (usernameField) {{
        console.log('Username field found:', usernameField);
        usernameField.value = '{username}';
        console.log('Username field value set.');
    }} else {{
        console.error('Username field not found for {site_url_base}');
    }}

    if (passwordField) {{
        console.log('Password field found:', passwordField);
        passwordField.value = '{password}';
        console.log('Password field value set.');
    }} else {{
        console.error('Password field not found for {site_url_base}');
    }}

    if (loginButton) {{
        console.log('Login button found:', loginButton);
        loginButton.click(); // 또는 loginButton.form.submit();
        console.log('Login button clicked.');
    }} else {{
        console.warn('Login button not found for {site_url_base}. Form submission might be required.');
    }}
}})();
            """)
    domesin_script_path = os.path.join("site_scripts", "www-domesin-co-kr.js")
    if not os.path.exists(domesin_script_path):
        with open(domesin_script_path, "w", encoding="utf-8") as f:
            f.write("""
(function() {{
    let usernameField = document.querySelector('{username_selector}'); // 실제 선택자로 변경
    let passwordField = document.querySelector('{password_selector}'); // 실제 선택자로 변경
    let loginButton = document.querySelector('{login_button_selector}'); // 실제 선택자로 변경

    console.log('Login attempt: {login_attempt_count}');
    console.log('Username field selector: {username_selector}, Password field selector: {password_selector}, Login button selector: {login_button_selector}');

    if (usernameField) {{
        console.log('Username field found:', usernameField);
        usernameField.value = '{username}';
        console.log('Username field value set.');
    }} else {{
        console.error('Username field not found for {site_url_base}');
    }}

    if (passwordField) {{
        console.log('Password field found:', passwordField);
        passwordField.value = '{password}';
        console.log('Password field value set.');
    }} else {{
        console.error('Password field not found for {site_url_base}');
    }}

    if (loginButton) {{
        console.log('Login button found:', loginButton);
        loginButton.click(); // 또는 loginButton.form.submit(); 폼 제출 방식이 더 안정적일 수 있습니다.
        console.log('Login button clicked.');
    }} else {{
        console.warn('Login button not found for {site_url_base}. Form submission might be required.');
    }}
}})();
            """)

    # site_url_patterns.json 파일 생성 (없을 경우) - 예시 내용 업데이트
    site_url_patterns_path = "site_url_patterns.json"
    if not os.path.exists(site_url_patterns_path):
        with open(site_url_patterns_path, "w", encoding="utf-8") as f:
            json.dump({
                  "www.dzb.co.kr": {
                    "category_list_url": "https://www.dzb.co.kr/goods/goods_list.php?cateCd={category_code}",
                    "page_url_format": "https://www.dzb.co.kr/goods/goods_list.php?page={page_number}&cateCd={category_code}",
                    "start_page": 1,
                    "page_increment": 1,
                    "start_category_code": "106",
                    "product_link_selectors": [
                        'ul.prdList > li a[href]',
                        '.prd-item a[href]',
                        'div.list a[href]' # Added div.list selector which was working in logs
                    ],
                    "next_page_selectors": [] # Next page selectors are no longer used, can be empty
                  },
                  "www.domesin.co.kr": {
                    "category_list_url": "https://domesin.co.kr/product/list.html?cate_no={category_code}&page=1", # 시작 페이지 명시
                    "page_url_format": "https://domesin.co.kr/product/list.html?cate_no={category_code}&page={page_number}",
                    "start_page": 1,
                    "page_increment": 1,
                    "start_category_code": "100", # 다른 카테고리 코드 예시
                    "product_link_selectors": [
                        'ul.prdList > li a[href]',
                        'li.item a[href]', # 예시 추가
                        'div.list a[href]' # 예시 추가
                    ],
                    "next_page_selectors": [
                        'a.page-next',
                        '.pagination > li:last-child > a' # 예시 추가
                    ],
                    "page_number_param": "page" # 페이지 번호 파라미터 이름 명시
                  },
                  "another-site.com": {
                    "category_list_url": "https://www.another-site.com/category/{category_code}",
                    "page_url_format": "https://www.another-site.com/category/{category_code}/page/{page_number}",
                    "start_page": 1,
                    "page_increment": 1,
                    "start_category_code": "default-category",
                    "product_link_selectors": [
                        'a.product-link[href]'
                    ],
                    "next_page_selectors": [
                        'a.next-page-button'
                    ]
                  }
                }, f, indent=2, ensure_ascii=False)

    main()

2025-02-20 09:49:44,588 - INFO - Loading site: https://www.domesin.co.kr


JavaScript console message: level=2, message='Uncaught ReferenceError: AOS is not defined', lineNumber=6821, sourceId='https://www.domesin.co.kr/'
JavaScript console message: level=1, message="Synchronous XMLHttpRequest on the main thread is deprecated because of its detrimental effects to the end user's experience. For more help, check https://xhr.spec.whatwg.org/.", lineNumber=4, sourceId='https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js'


2025-02-20 09:49:47,869 - INFO - Page loaded successfully: https://www.domesin.co.kr/
2025-02-20 09:49:51,566 - INFO - Loading site: https://www.funtasticb2b.co.kr


JavaScript console message: level=2, message="Mixed Content: The page at 'https://www.funtasticb2b.co.kr/' was loaded over HTTPS, but requested an insecure stylesheet 'http://fonts.googleapis.com/css?family=Noto+Sans'. This request has been blocked; the content must be served over HTTPS.", lineNumber=0, sourceId='https://www.funtasticb2b.co.kr/'


2025-02-20 09:49:52,936 - INFO - Page loaded successfully: https://www.funtasticb2b.co.kr/
2025-02-20 09:49:56,043 - INFO - Loading site: https://www.osong.shop


JavaScript console message: level=1, message="Unrecognized feature: 'attribution-reporting'.", lineNumber=290, sourceId='https://connect.facebook.net/en_US/fbevents.js'


2025-02-20 09:49:58,057 - INFO - Page loaded successfully: https://www.osong.shop/
